## Extracting colors from an image

In [ ]:
from google.colab import files
uploaded_files = files.upload()

In [ ]:
import plotly.express as px
import math
import cv2

SRC_IMAGE = 'sample-3.jpg' # Image path
MAX_DIM = 100 # Set maximum width/height of the image

img = cv2.imread(SRC_IMAGE)
print('Original Dimensions (Height,Width,Colors):', img.shape)

# image dimensions order: Height, Width, Depth
width = 0
height = 0
if img.shape[1] > MAX_DIM or img.shape[0] > MAX_DIM:
  if img.shape[1] > img.shape[0]:
    # Width > Height
    width = MAX_DIM
    height = math.floor(width / img.shape[1] * img.shape[0])
  else:
    # Height >= Width
    height = MAX_DIM
    width = math.floor(height / img.shape[0] * img.shape[1])

  dim = (width, height)

  # downscale/compress image
  img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
  print('Resized Dimensions (Height,Width,Colors):', img.shape)

img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

fig = px.imshow(img)
fig.show()

In [ ]:
from sklearn.cluster import KMeans

class DominantColors:

  CLUSTERS = None
  MAX_DIM = None
  IMAGE = None
  COLORS = None
  LABELS = None

  def __init__(self, image, clusters=7, max_dim=500):
    self.CLUSTERS = clusters
    self.IMAGE = image
    self.MAX_DIM = max_dim

  def dominantColors(self):
    img = cv2.imread(self.IMAGE)

    # Downscale/resize image
    # image dimensions order: Height, Width, Depth
    if img.shape[1] > MAX_DIM or img.shape[0] > MAX_DIM:
      if img.shape[1] > img.shape[0]:
        # Width > Height
        width = MAX_DIM
        height = math.floor(width / img.shape[1] * img.shape[0])
      else:
        # Height >= Width
        height = MAX_DIM
        width = math.floor(height / img.shape[0] * img.shape[1])

      dim = (width, height)
      img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # reshaping to a list of pixel [r,g,b] values
    img = img.reshape((img.shape[1] * img.shape[0], 3)) # Width * Height, 3
    self.IMAGE = img

    # using k-means to cluster pixels
    kmeans = KMeans(n_clusters = self.CLUSTERS, n_init='auto')
    kmeans.fit(img)

    # cluster centers are dominant colors
    self.COLORS = kmeans.cluster_centers_.astype(int)

    # save cluster labels for each pixel
    self.LABELS = kmeans.labels_

    return self.COLORS

  def rgb_to_hex(self, rgb):
    return '#{:02x}{:02x}{:02x}'.format(rgb[0], rgb[1], rgb[2])

  def plotClusters(self):
    X, Y, Z = zip(*self.IMAGE) # convert pixels into [Reds, Greens, Blues] array

    Colors = []
    for label in self.LABELS:
      Colors.append(self.rgb_to_hex(self.COLORS[label]))

    fig = px.scatter_3d(x=X, y=Y, z=Z, color=Colors,
                        color_discrete_map=dict(zip(Colors, Colors)),
                        labels={
                            'x': 'Reds',
                            'y': 'Greens',
                            'z': 'Blues',
                        },
                        title='Dominant colors clusters')
    fig.show()

In [ ]:
dc = DominantColors(SRC_IMAGE)
colors = dc.dominantColors()
hex_colors = []
for c in colors:
  hex_colors.append(dc.rgb_to_hex(c))
print(colors)
print(hex_colors)
dc.plotClusters()

## Making a color palette from a single color

Convert color to HSV format for manipulation of hue, brightness and saturation.

In [ ]:
import colorsys
# ColorSys uses values in fraction i.e. [0, 1]
hls = []
for color in colors:
  h, l, s = colorsys.rgb_to_hls(color[0]/255, color[1]/255, color[2]/255)
  # convert hls back to original values from 0, 1
  h = round(h * 360) # h ranges from [0, 360]
  l = round(l * 100) # l is a percent value [0, 100]
  s = round(s * 100) # s is a percent value [0, 100]
  # there will be some loss in converting float to int
  hls.append([h, l, s])
print(hls)

### Step 1: Find complimentry color to the Primary Color
If Hue (H) value < 180° add 180 to it. If it's > 180° then subtract 180 from it. This will give you the exact complement of a color. (Basically we are rotating color wheel by half turn.) \
Secondary color should not overpower primary color, so mute the complementary color by lowering it's brightness and saturation.

In [ ]:
# let primary color be
primary_color = hls[-1]
secondary_color = [0, 0, 0]
if primary_color[0] < 180:
  secondary_color[0] = 180 + primary_color[0]
else:
  secondary_color[0] = primary_color[0] - 180

# reducing brightness by 25%
secondary_color[1] = round(primary_color[1] * 0.75)

# reducing saturation by 30%
secondary_color[2] = round(primary_color[2] * 0.3)
print(secondary_color)

### Step 2: Make tints for each color
Duplicate the shades for each color and then make the color brighter and less saturated.

In [ ]:
primary_tint = primary_color.copy()
secondary_tint = secondary_color.copy()

# brightening by 75%
primary_tint[1] = round(primary_tint[1] * 1.75)
secondary_tint[1] = round(secondary_tint[1] * 1.75)

# desaturating by 75%
primary_tint[2] = round(primary_tint[2] * 0.25)
secondary_tint[2] = round(secondary_tint[2] * 0.25)

print(primary_tint, secondary_tint)

### Step 3: Make shadows of primary and secondary colors
Reduce the brightness by about 50% of primary and secondary shades. \
Shift the hues of shadows towards bluw and make them slightly more saturated. \
Blue = (R: 0, G: 0, B: 255) = (H: 240, L: 50%, S: 100%)

In [ ]:
primary_shadow = primary_color.copy()
secondary_shadow = secondary_color.copy()

# reducing brightness by 50%
primary_shadow[1] = round(primary_shadow[1] * 0.5)
secondary_shadow[1] = round(secondary_shadow[1] * 0.5)

# shifting hue towards blue by 20 points
if primary_shadow[0] > 240:
  primary_shadow[0] -= 20
else:
  primary_shadow[0] += 20
if secondary_shadow[0] > 240:
  secondary_shadow[0] -= 20
else:
  secondary_shadow[0] += 20

# increasing saturation by 20%
primary_shadow[2] = round(primary_shadow[2] * 1.2)
secondary_shadow[2] = round(secondary_shadow[2] * 1.2)

print(primary_shadow, secondary_shadow)

In [ ]:
ColorScheme = [primary_color, secondary_color, primary_tint, secondary_tint, primary_shadow, secondary_shadow]
for idx, swatch in enumerate(ColorScheme):
  r, g, b = colorsys.hls_to_rgb(swatch[0]/360, swatch[1]/100, swatch[2]/100)
  ColorScheme[idx] = dc.rgb_to_hex([int(r*255), int(g*255), int(b*255)])

print(ColorScheme)

### References

Procedure:
- https://medium.com/@ys3372/deconstructing-an-image-with-pixels-4c65c3a2268c
- https://theschedio.com/make-your-color-palette/

Color theory:
- https://donatbalipapp.medium.com/colours-maths-90346fb5abda
- https://www.youtube.com/watch?v=j17KACvRS1o
- https://manifold.net/doc/mfd8/colors_as_hue_saturation_and_brightness.htm
- https://changingminds.org/explanations/perception/visual/hsl.htm